# Analysis of integrated models in MEWpy

MEWpy supports several methods to perform phenotype simulations using integrated MEW models.
The following simulation methods are available in `mewpy.mew.analysis`:
- `FBA` - requires a Metabolic model
- `pFBA` - requires a Metabolic model
- `RFBA` - requires a Regulatory-Metabolic model
- `SRFBA` - requires a Regulatory-Metabolic model
- `PROM` - requires a Regulatory-Metabolic model
- `CoRegFlux` - requires a Regulatory-Metabolic model

In addition, `FBA` and `pFBA` simulation methods are available in the MEWpy `Simulator` object.

This example uses the integrated _E. coli_ core model published by [Orth _et al_, 2010](https://doi.org/10.1128/ecosalplus.10.2.1). More information regarding this model is available in `examples.mew_models.ipynb` notebook

This example uses the integrated _E. coli_ iMC1010 model published by [Covert _et al_, 2004](https://doi.org/10.1038/nature02456). This model consists of the _E. coli_ iJR904 GEM model published by [Reed _et al_, 2003](https://doi.org/10.1186/gb-2003-4-9-r54) and _E. coli_ iMC1010 TRN published by [Covert _et al_, 2004](https://doi.org/10.1038/nature02456). This model includes 904 metabolic genes, 931 unique biochemical reactions, and a TRN having 1010 regulatory interactions (target-regulators using boolean logic).

This example uses the integrated _M. tuberculosis_ iNJ661 model published by [Chandrasekaran _et al_, 2010](https://doi.org/10.1073/pnas.1005139107). This model consists of the _M. tuberculosis_ iNJ661 GEM model published by [Jamshidi _et al_, 2007](https://doi.org/10.1186/1752-0509-1-26), _M. tuberculosis_ TRN published by [Balazsi _et al_, 2008](https://doi.org/10.1038/msb.2008.63), and gene expression dataset published by [Chandrasekaran _et al_, 2010](https://doi.org/10.1073/pnas.1005139107). This model includes 691 metabolic genes, 1028 unique biochemical reactions, and a TRN having 2018 regulatory interactions (target-regulator).

This example uses the integrated _S. cerevisae_ iMM904 model published by [Banos _et al_, 2017](https://doi.org/10.1186/s12918-017-0507-0). This model consists of the _S. cerevisae_ iMM904 GEM model published by [Mo _et al_, 2009](https://doi.org/10.1186/1752-0509-3-37), _S. cerevisae_ TRN inferred by CoRegNet published by [Nicolle _et al_, 2015](https://doi.org/10.1093/bioinformatics/btv305), and gene expression datasets published by [Brauer _et al_, 2005](https://doi.org/10.1091/mbc.e04-11-0968) and [DeRisi _et al_, 1997](https://doi.org/10.1126/science.278.5338.680). This model includes 904 metabolic genes, 1557 unique biochemcial reactions, and a TRN having 3748 regulatory interactions (target-regulators separated in co-activators and co-repressors).

In [1]:
# imports
import os
from pathlib import Path

from mewpy.io import read_model, Engines, Reader, read_model

In [2]:
# readers
path = Path(os.getcwd()).joinpath('models', 'regulation')

# E. coli core
core_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('e_coli_core.xml'))
core_trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                         path.joinpath('e_coli_core_trn.csv'),
                         sep=',',
                         id_col=0,
                         rule_col=2,
                         aliases_cols=[1],
                         header=0)

# E. coli iMC1010
imc1010_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iJR904_srfba.xml'))
imc1010_trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                            path.joinpath('iMC1010.csv'),
                            sep=',',
                            id_col=0,
                            rule_col=4,
                            aliases_cols=[1, 2, 3],
                            header=0)

# M. tuberculosis iNJ661
inj661_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iNJ661.xml'))
inj661_trn_reader = Reader(Engines.CoExpressionRegulatoryCSV,
                           path.joinpath('iNJ661_trn.csv'),
                           sep=',',
                           target_col=2,
                           co_activating_col=3,
                           co_repressing_col=4,
                           header=0)

# S. cerevisae iMM904
imm904_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iMM904.xml'))
imm904_trn_reader = Reader(Engines.TargetRegulatorRegulatoryCSV,
                           path.joinpath('iMM904_trn.csv'),
                           sep=';',
                           target_col=0,
                           regulator_col=1,
                           header=None)

## Working with phenotype simulation in MEWpy
In the `mew.analysis` package, simulation methods are derived from a `LinearProblem` object having the following attributes and methods:
- `method` - the name of the simulation method
- `model` - the model used to build the linear problem
- `solver` - a MEWpy solver instance having the linear programming implementation of variables and constraints in the selected solver. The following solvers are available: _CPLEX_; _GUROBI_; _OPTLANG_
- `constraints` - The representation of ODE to be implemented in the solver instance using linear programming
- `variables` - The representation of the system variables to be implemented in the solver instance using linear programming
- `objective` - A linear representation of the objective function associated with the linear problem

A simulation method includes two important methods:
- `build` - the build method is responsible for retrieving variables and constraints from a MEW model according to the mathematical formulation of each simulation method
- `optimize` - the optimize method is responsible for solving the linear problem using linear programming or mixed-integer linear programming. This method accepts method-specific arguments (initial state, dynamic, etc) and solver-specific arguments (linear, minimize, constraints, get_values, etc). These arguments can override temporarily some constraints or variables during the optimization.

In [5]:
# imports
from mewpy.mew.analysis import *

In [19]:
# showcase of a simulation method

# reading the E. coli core model
model = read_model(core_gem_reader, core_trn_reader)

# initialization does not build the model automatically
srfba = SRFBA(model).build()
srfba

Method,SRFBA
Model,Model e_coli_core_trn - model
Variables,486
Constraints,326
Objective,{'Biomass_Ecoli_core': 1.0}
Solver,CplexSolver
Synchronized,True


The `optimize` interface creates a `ModelSolution` output by default containing the objective value, value of each variable in the solution, among others. Alternatively, `optimize` can create a simple solver `Solution` object.

In [20]:
# optimization creates a ModelSolution object by default
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core_trn - model
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684829
Status,optimal


One can generate a pandas `DataFrame` using the `to_frame()` method of the `ModelSolution` object.
This data frame contains the obtained expression coefficients for the regulatory environmental stimuli linked to the metabolic model and exchange fluxes.

In [21]:
# a solution can be converted into a df
solution.to_frame()

regulatory                         \
                   regulatory variable          variable type   
Biomass_Ecoli_core  Biomass_Ecoli_core    reaction, regulator   
FBP                                FBP    reaction, regulator   
GLCpts                          GLCpts    reaction, regulator   
LDH_D                            LDH_D    reaction, regulator   
ME1                                ME1    reaction, regulator   
ME2                                ME2    reaction, regulator   
PFK                                PFK    reaction, regulator   
PGI                                PGI    reaction, regulator   
PYK                                PYK    reaction, regulator   
SUCCt2_2                      SUCCt2_2    reaction, regulator   
TALA                              TALA    reaction, regulator   
TKT2                              TKT2    reaction, regulator   
pi_e                              pi_e  metabolite, regulator   
glc__D_e                      glc__D_e  metabolite, regulator   
ac_e                              ac_e  metabolite, regulator   
o2_e                              o2_e  metabolite, regulator   
nh4_e                            nh4_e  metabolite, regulator   
glu__L_e                      glu__L_e  metabolite, regulator   
succ_e                          succ_e  metabolite, regulator   
fum_e                            fum_e  metabolite, regulator   
mal__L_e                      mal__L_e  metabolite, regulator   
lac__D_e                      lac__D_e  metabolite, regulator   
fru_e                            fru_e  metabolite, regulator   
EX_ac_e                            NaN                    NaN   
EX_acald_e                         NaN                    NaN   
EX_akg_e                           NaN                    NaN   
EX_co2_e                           NaN                    NaN   
EX_etoh_e                          NaN                    NaN   
EX_for_e                           NaN                    NaN   
EX_fru_e                           NaN                    NaN   
EX_fum_e                           NaN                    NaN   
EX_glc__D_e                        NaN                    NaN   
EX_gln__L_e                        NaN                    NaN   
EX_glu__L_e                        NaN                    NaN   
EX_h_e                             NaN                    NaN   
EX_h2o_e                           NaN                    NaN   
EX_lac__D_e                        NaN                    NaN   
EX_mal__L_e                        NaN                    NaN   
EX_nh4_e                           NaN                    NaN   
EX_o2_e                            NaN                    NaN   
EX_pi_e                            NaN                    NaN   
EX_pyr_e                           NaN                    NaN   
EX_succ_e                          NaN                    NaN   

                                                            \
                   minimum coefficient maximum coefficient   
Biomass_Ecoli_core                 0.0              1000.0   
FBP                                0.0              1000.0   
GLCpts                             0.0              1000.0   
LDH_D                          -1000.0              1000.0   
ME1                                0.0              1000.0   
ME2                                0.0              1000.0   
PFK                                0.0              1000.0   
PGI                            -1000.0              1000.0   
PYK                                0.0              1000.0   
SUCCt2_2                           0.0              1000.0   
TALA                           -1000.0              1000.0   
TKT2                           -1000.0              1000.0   
pi_e                           -1000.0              1000.0   
glc__D_e                         -10.0              1000.0   
ac_e                               0.0              1000.0   
o2_e                           -1000.0          

One can generate a `Summary` object using the `to_summary()` method of the `ModelSolution` object.
This summary contains the following data:
- inputs - regulatory and metabolic inputs for the simulation method
- outputs - regulatory and metabolic inputs for the simulation method
- metabolic - values of the metabolic variables
- regulatory - values of the regulatory variables
- objective - the objective value
- df - the summary of inputs and outputs in the regulatory and metabolic layers

In [22]:
# a solution can be converted into a summary solution
summary = solution.to_summary()
summary

In [28]:
# inputs + outputs of the metabolic-regulatory variables
summary.df

regulatory                              \
                   regulatory variable        variable type   role   
Biomass_Ecoli_core  Biomass_Ecoli_core  reaction, regulator  input   
FBP                                FBP  reaction, regulator  input   
GLCpts                          GLCpts  reaction, regulator  input   
LDH_D                            LDH_D  reaction, regulator  input   
ME1                                ME1  reaction, regulator  input   
...                                ...                  ...    ...   
EX_h_e                             NaN                  NaN    NaN   
EX_h2o_e                           NaN                  NaN    NaN   
EX_nh4_e                           NaN                  NaN    NaN   
EX_o2_e                            NaN                  NaN    NaN   
EX_pi_e                            NaN                  NaN    NaN   

                                          metabolic                           \
                   expression coefficient  reaction variable type metabolite   
Biomass_Ecoli_core           8.739215e-01       NaN           NaN        NaN   
FBP                          0.000000e+00       NaN           NaN        NaN   
GLCpts                       1.000000e+01       NaN           NaN        NaN   
LDH_D                        2.273737e-13       NaN           NaN        NaN   
ME1                          0.000000e+00       NaN           NaN        NaN   
...                                   ...       ...           ...        ...   
EX_h_e                                NaN    EX_h_e      reaction        h_e   
EX_h2o_e                              NaN  EX_h2o_e      reaction      h2o_e   
EX_nh4_e                              NaN  EX_nh4_e      reaction      nh4_e   
EX_o2_e                               NaN   EX_o2_e      reaction       o2_e   
EX_pi_e                               NaN   EX_pi_e      reaction       pi_e   

                                       
                      role       flux  
Biomass_Ecoli_core     NaN        NaN  
FBP                    NaN        NaN  
GLCpts                 NaN        NaN  
LDH_D                  NaN        NaN  
ME1                    NaN        NaN  
...                    ...        ...  
EX_h_e              output  17.530865  
EX_h2o_e            output  29.175827  
EX_nh4_e             input  -4.765319  
EX_o2_e              input -21.799493  
EX_pi_e              input  -3.214895  

[189 rows x 9 columns]

In [23]:
# values of the metabolic variables
summary.metabolic

,reaction,variable type,metabolite,role,flux
EX_co2_e,EX_co2_e,reaction,co2_e,output,22.809833
EX_glc__D_e,EX_glc__D_e,reaction,glc__D_e,input,-10.000000
EX_h_e,EX_h_e,reaction,h_e,output,17.530865
EX_h2o_e,EX_h2o_e,reaction,h2o_e,output,29.175827
EX_nh4_e,EX_nh4_e,reaction,nh4_e,input,-4.765319
EX_o2_e,EX_o2_e,reaction,o2_e,input,-21.799493
EX_pi_e,EX_pi_e,reaction,pi_e,input,-3.214895


In [24]:
# values of the regulatory variables
summary.regulatory

,regulatory variable,variable type,role,expression coefficient
Biomass_Ecoli_core,Biomass_Ecoli_core,"reaction, regulator",input,8.739215e-01
FBP,FBP,"reaction, regulator",input,0.000000e+00
GLCpts,GLCpts,"reaction, regulator",input,1.000000e+01
LDH_D,LDH_D,"reaction, regulator",input,2.273737e-13
ME1,ME1,"reaction, regulator",input,0.000000e+00
...,...,...,...,...
CRPnoGLM,CRPnoGLM,"target, regulator",output,1.000000e+00
NRI_hi,NRI_hi,"target, regulator",output,0.000000e+00
NRI_low,NRI_low,"target, regulator",output,-0.000000e+00
surplusFDP,surplusFDP,"target, regulator",output,0.000000e+00


In [25]:
# objective value
summary.objective

,value,direction
Biomass_Ecoli_core,0.873922,maximize


In [26]:
# values of the metabolic and regulatory inputs
summary.inputs

regulatory                         \
                             regulator          variable type   
Biomass_Ecoli_core  Biomass_Ecoli_core    reaction, regulator   
FBP                                FBP    reaction, regulator   
GLCpts                          GLCpts    reaction, regulator   
LDH_D                            LDH_D    reaction, regulator   
ME1                                ME1    reaction, regulator   
ME2                                ME2    reaction, regulator   
PFK                                PFK    reaction, regulator   
PGI                                PGI    reaction, regulator   
PYK                                PYK    reaction, regulator   
SUCCt2_2                      SUCCt2_2    reaction, regulator   
TALA                              TALA    reaction, regulator   
TKT2                              TKT2    reaction, regulator   
pi_e                              pi_e  metabolite, regulator   
glc__D_e                      glc__D_e  metabolite, regulator   
ac_e                              ac_e  metabolite, regulator   
o2_e                              o2_e  metabolite, regulator   
nh4_e                            nh4_e  metabolite, regulator   
glu__L_e                      glu__L_e  metabolite, regulator   
succ_e                          succ_e  metabolite, regulator   
fum_e                            fum_e  metabolite, regulator   
mal__L_e                      mal__L_e  metabolite, regulator   
lac__D_e                      lac__D_e  metabolite, regulator   
fru_e                            fru_e  metabolite, regulator   
EX_glc__D_e                        NaN                    NaN   
EX_nh4_e                           NaN                    NaN   
EX_o2_e                            NaN                    NaN   
EX_pi_e                            NaN                    NaN   

                                             metabolic                \
                   expression coefficient     reaction variable type   
Biomass_Ecoli_core           8.739215e-01          NaN           NaN   
FBP                          0.000000e+00          NaN           NaN   
GLCpts                       1.000000e+01          NaN           NaN   
LDH_D                        2.273737e-13          NaN           NaN   
ME1                          0.000000e+00          NaN           NaN   
ME2                          0.000000e+00          NaN           NaN   
PFK                          7.477382e+00          NaN           NaN   
PGI                          4.860861e+00          NaN           NaN   
PYK                          1.758177e+00          NaN           NaN   
SUCCt2_2                     0.000000e+00          NaN           NaN   
TALA                         1.496984e+00          NaN           NaN   
TKT2                         1.181498e+00          NaN           NaN   
pi_e                         1.000000e-10          NaN           NaN   
glc__D_e                     1.000000e-10          NaN           NaN   
ac_e                         1.000000e-10          NaN           NaN   
o2_e                         1.000000e+00          NaN           NaN   
nh4_e                        1.000000e+03          NaN           NaN   
glu__L_e                     1.000000e-10          NaN           NaN   
succ_e                       1.000000e+03          NaN           NaN   
fum_e                        1.000000e-10          NaN           NaN   
mal__L_e                     1.000000e-10          NaN           NaN   
lac__D_e                     1.000000e-10          NaN           NaN   
fru_e                        1.000000e-10          NaN           NaN   
EX_glc__D_e                           NaN  EX_glc__D_e      reaction   
EX_nh4_e                              NaN     EX_nh4_e      reaction   
EX_o2_e                               NaN      EX_o2_e      reaction   
EX_pi_e                               NaN      EX_pi_e      reaction   

                                          
          

In [27]:
# values of the metabolic and regulatory outputs
summary.outputs

regulatory                                           metabolic  \
                target      variable type expression coefficient  reaction   
b0008            b0008       target, gene                    1.0       NaN   
b0080            b0080  target, regulator                    1.0       NaN   
b0113            b0113  target, regulator                    1.0       NaN   
b0114            b0114       target, gene                    1.0       NaN   
b0115            b0115       target, gene                    1.0       NaN   
...                ...                ...                    ...       ...   
surplusFDP  surplusFDP  target, regulator                    0.0       NaN   
surplusPYR  surplusPYR  target, regulator                    0.0       NaN   
EX_co2_e           NaN                NaN                    NaN  EX_co2_e   
EX_h_e             NaN                NaN                    NaN    EX_h_e   
EX_h2o_e           NaN                NaN                    NaN  EX_h2o_e   

                                                
           variable type metabolite       flux  
b0008                NaN        NaN        NaN  
b0080                NaN        NaN        NaN  
b0113                NaN        NaN        NaN  
b0114                NaN        NaN        NaN  
b0115                NaN        NaN        NaN  
...                  ...        ...        ...  
surplusFDP           NaN        NaN        NaN  
surplusPYR           NaN        NaN        NaN  
EX_co2_e        reaction      co2_e  22.809833  
EX_h_e          reaction        h_e  17.530865  
EX_h2o_e        reaction      h2o_e  29.175827  

[162 rows x 7 columns]

## Working with MEW model and phenotype simulation in MEWpy
A phenotype simulation method must be initialized with a MEW model. A common workflow to work with MEW models and simulation methods is suggested as follows:
1. `model = read_model(reader1, reader2)` - read the model
2. `rfba = RFBA(model)` - initialize the simulation method
3. `rfba.build()` - build the linear problem
4. `solution = rfba.optimize()` - perform the optimization
5. `model.reactions['MY_REACTION'].bounds = (0, 0)` - make changes to the model
6. `solution = RFBA(model).build().optimize()` - initialize, build and optimize the simulation method

In this workflow, _model_ and _rfba_ instances are not connected with each other. Future rfba's optimization will generate the same output even if we make changes to the model. That is, _model_ and _rfba_ are not synchronized and attached to each other.
<br>

Although building linear problems is considerably fast for most models, there is a second workflow to work with MEW models and simulation methods:
1. `model = read_model(reader1, reader2)` - read the model
2. `rfba = RFBA(model, attach=True)` - initialize the simulation method and attach it to the model
3. `rfba.build()` - build the linear problem
4. `solution = rfba.optimize()` - perform the optimization
5. `model.reactions['MY_REACTION'].bounds = (0, 0)` - make changes to the model
6. `rxn_ko_solution = rfba.optimize()` - perform the optimization again but this time with the reaction deletion

In [30]:
# read, build, optimize
model = read_model(core_gem_reader, core_trn_reader)
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core_trn - model
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684829
Status,optimal


In [38]:
# make changes and then build, optimize
model.regulators['b3261'].ko()
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core_trn - model
Objective,Biomass_Ecoli_core
Objective value,0.0
Status,optimal


In [40]:
# second workflow
model = read_model(core_gem_reader, core_trn_reader)
srfba = SRFBA(model, attach=True).build()
solution = srfba.optimize()
print('Wild-type growth rate', solution.objective_value)

# applying the knockout
model.regulators['b3261'].ko()
solution = srfba.optimize()
print('KO growth rate', solution.objective_value)

Wild-type growth rate 0.8739215069684829
KO growth rate 0.0


In addition, one can attach as many simulation methods as needed to a single model instance. This behavior eases the comparison between simulation methods

In [44]:
# many simulation methods attached
model = read_model(core_gem_reader, core_trn_reader)
fba = FBA(model, attach=True).build()
pfba = pFBA(model, attach=True).build()
rfba = RFBA(model, attach=True).build()
srfba = SRFBA(model, attach=True).build()

# applying the knockout
model.regulators['b3261'].ko()

print('FBA KO growth rate:', fba.optimize().objective_value)
print('pFBA KO sum of fluxes:', pfba.optimize().objective_value)
print('RFBA KO growth rate:', rfba.optimize().objective_value)
print('SRFBA KO growth rate:', srfba.optimize().objective_value)
print()

# restore the model
model.undo()
print('FBA WT growth rate:', fba.optimize().objective_value)
print('pFBA WT sum of fluxes:', pfba.optimize().objective_value)
print('RFBA WT growth rate:', rfba.optimize().objective_value)
print('SRFBA WT growth rate:', srfba.optimize().objective_value)

FBA KO growth rate: 0.8739215069684303
pFBA KO sum of fluxes: 93768.8478640836
RFBA KO growth rate: 0.0
SRFBA KO growth rate: 0.0

FBA WT growth rate: 0.8739215069684303
pFBA WT sum of fluxes: 93768.8478640836
RFBA WT growth rate: 0.0
SRFBA WT growth rate: 0.8739215069684829


## FBA and pFBA

MEWpy supports FBA and pFBA simulation methods using MEW models.
<br>
FBA is a phenotype simulation method based on mass balance constraints retrieved from metabolites and reactions found in a GEM model. FBA is aimed at finding the maximum value for the objective function. As the biomass reaction is often used as objective function, FBA is often used to find the optimal growth rate of an organism. For more details consult: [https://doi.org/10.1038/nbt.1614](https://doi.org/10.1038/nbt.1614). In addition, `mewpy.mew.analysis.FBA` also takes into consideration the coefficients of metabolic genes, thus limiting reactions bounds to the corresponding gene states.
<br>
pFBA is a phenotype simulation method based on FBA, as this method also finds the optimal growth rate. However, the objective function of pFBA consists of minimizing the total sum of all fluxes, and thus finding the subset of genes and proteins that may contribute to the most efficient metabolic network topology [Lewis _et al_, 2010](https://doi.org/10.1038/msb.2010.47).
<br>
`FBA` and `pFBA` are both available in the `mewpy.mew.analysis` package. Alternatively, one can use the simple and optimized versions `slim_fba` and `slim_pfba`. Likewise, `FBA` and `pFBA` are available in MEWpy's `Simulator`, which is the common interface to perform simulations using MEW models, COBRApy models, and Reframed models.

In [45]:
# using FBA analysis
met_model = read_model(core_gem_reader)
FBA(met_model).build().optimize()

Method,FBA
Model,Model e_coli_core - model
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684303
Status,optimal


In [48]:
# using slim FBA analysis
slim_fba(met_model)

0.8739215069684303

In [51]:
# using MEWpy simulator
from mewpy.simulation import get_simulator
simulator = get_simulator(met_model)
simulator.simulate()

objective: 0.8739215069684303
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA

In [53]:
from mewpy.simulation import SimulationMethod

# pfba version
print(pFBA(met_model).build().optimize().objective_value)
print(slim_pfba(met_model))
print(simulator.simulate(method=SimulationMethod.pFBA))

93768.8478640836
93768.8478640836
objective: 0.8739215069684303
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.pFBA


## FVA and Deletions
The `mewpy.mew.analysis` package includes the FVA method to inspect the solution space of a GEM model.
FVA computes the minimum and maximum possible fluxes of each reaction in a metabolic model. This method can be used to identify reactions limiting cellular growth. This method return a pandas `DataFrame` with the minium and maximum fluxes (columns) for each reaction (index).
<br>
The `mewpy.mew.analysis` package includes `single_gene_deletion` and `single_reaction_deletion` methods to inspect _in silico_ genetic strategies. These methods perform an FBA phenotype simulation of a single reaction deletion or gene knockout for all reactions and genes in the metabolic model. These methods are faster than iterating through the model reactions or genes using the `ko()` method.

In [55]:
# FVA returns the DataFrame with minium and maximum values of each reaction
fva(met_model)

,minimum,maximum
ACALD,-7.105427e-15,0.000000
ACALDt,0.000000e+00,0.000000
ACKr,0.000000e+00,0.000000
ACONTa,6.007250e+00,6.007250
ACONTb,6.007250e+00,6.007250
...,...,...
TALA,1.496984e+00,1.496984
THD2,0.000000e+00,0.000000
TKT1,1.496984e+00,1.496984
TKT2,1.181498e+00,1.181498


In [57]:
# FVA with the first five reactions only
reactions_ids = list(met_model.reactions)[:5]
fva(met_model, reactions=reactions_ids)

,minimum,maximum
ACALD,-7.105427e-15,0.00000
ACALDt,0.000000e+00,0.00000
ACKr,0.000000e+00,0.00000
ACONTa,6.007250e+00,6.00725
ACONTb,6.007250e+00,6.00725


In [59]:
# single reaction deletion
single_reaction_deletion(met_model)

,growth,status
ACALD,0.873922,Optimal
ACALDt,0.873922,Optimal
ACKr,0.873922,Optimal
ACONTa,0.000000,Optimal
ACONTb,0.000000,Optimal
...,...,...
TALA,0.864759,Optimal
THD2,0.873922,Optimal
TKT1,0.864759,Optimal
TKT2,0.866674,Optimal


In [60]:
# single gene deletion
single_gene_deletion(met_model)

,growth,status
b0351,0.873922,Optimal
b1241,0.873922,Optimal
s0001,0.211141,Optimal
b2296,0.873922,Optimal
b3115,0.873922,Optimal
...,...,...
b2464,0.873922,Optimal
b0008,0.873922,Optimal
b2935,0.873922,Optimal
b2465,0.873922,Optimal


In [63]:
# single gene deletion for specific genes
single_gene_deletion(met_model, genes=met_model.reactions['ACONTa'].genes)

,growth,status
b0118,0.873922,Optimal
b1276,0.873922,Optimal


## Regulatory Truth Table
The regulatory truth table of a regulatory model contains the evaluation of all regulatory interactions.
The `mewpy.mew.analysis.regulatory_truth_table` method creates the combination between the regulators and target genes given a regulatory model. This function returns a pandas `DataFrame` having the regulators' values in the columns and targets' outcome in the index.

In [5]:
# regulatory truth table for the regulatory model
reg_model = read_model(core_trn_reader)
regulatory_truth_table(reg_model)

,result,surplusFDP,surplusPYR,b0113,b3261,b0400,pi_e,b4401,b1334,b3357,...,TALA,PGI,fru_e,ME2,ME1,GLCpts,PYK,PFK,LDH_D,SUCCt2_2
b0008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0080,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0113,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0114,1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0115,1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRPnoGLM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NRI_hi,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NRI_low,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surplusFDP,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## RFBA
RFBA is a phenotype simulation method based on the integration of a GEM model with a TRN at the genome-scale. The TRN consists of a set of regulatory interactions formulated with boolean and propositional logic. The TRN contains a boolean algebra expression for each target gene. This boolean rule determines whether the target gene is active (1) or not (0) according to the state of the regulators (active or inactive). Then, the TRN is integrated with the GEM model using the reactions' GPR rules. It is also common to find metabolites and reactions as regulators/environmental stimuli in the TRN, completing the integration with the GEM model.

In RFBA, a synchronous evaluation of all regulatory interactions in the regulatory model is performed first. This first simulation is used to retrieve the regulatory state (regulators' coefficients). Then, the regulatory state is translated into a metabolic state (metabolic genes' coefficients) by performing another synchronous evaluation of all regulatory interactions in the regulatory model. Finally, the resulting metabolic state is used to decode the constraints imposed by the regulatory model upon evaluation of the reactions' GPRs with the targets' state.

RFBA supports steady-state or dynamic phenotype simulations. Dynamic RFBA simulation performs several sequential optimizations that update both the regulatory and metabolic constraints. This simulation stops when two identical solutions are found.

For more details consult: [https://doi.org/10.1038/nature02456](https://doi.org/10.1038/nature02456).

For this example we will be using iMC1010 model available at _models/regulation/iJR904_srfba.xml_ and _models/regulation/iMC1010.csv_

In [7]:
# loading model
model = read_model(imc1010_gem_reader, imc1010_trn_reader)

# objective function
BIOMASS_ID = 'BiomassEcoli'
model.objective = {BIOMASS_ID: 1}

# initial state inferred from the find_conflicts method
initial_state = {
    'Stringent': 0.0,
    'high-NAD': 0.0,
    'lys_DASH_L_e': 0.0,
    'phe_DASH_L_e': 0.0,
    'tyr_DASH_L_e': 0.0,
    'trp_DASH_L_e': 0.0,
    'leu_DASH_L_e': 0.0,
    'asn_DASH_L_e': 0.0,
    'glu_DASH_L_e': 0.0,
    'glcn_e': 0.0,
    'glc_DASH_D_e': 0.0,
    'arab_DASH_L_e': 0.0,
    'xyl_DASH_D_e': 0.0,
    'rib_DASH_D_e': 0.0,
    'mal_DASH_L_e': 0.0,
    'glyc_e': 0.0,
    'sbt_DASH_D_e': 0.0,
    'lac_DASH_D_e': 0.0,
    'hxan_e': 0.0,
    'gua_e': 0.0,
    'ura_e': 0.0,
    'csn_e': 0.0,
    'arg_DASH_L_e': 0.0,
    'met_DASH_L_e': 0.0,
    'cys_DASH_L_e': 0.0,
    'val_DASH_L_e': 0.0,
    'acgam_e': 0.0,
    'AGDC': 0.0,
}

RFBA can be simulated using an initial regulatory state. This initial state will be considered during the synchronous evaluation of all regulatory interactions in the regulatory model and determine the metabolic state. The set-up of the regulators' initial state in integrated models is a difficult task. Most of the time, the initial state is not known and hinders feasible solutions during simulation. Besides, if the initial state is not provided to RFBA, this method will consider that all regulators are active (inclusive metabolites and reactions). For large models, this clearly not the best initial state.
<br>
Hence, one should use the `mewpy.mew.analysis.find_conflicts()` method to ease the set-up of the initial state. This method can be used to find regulatory states that affect the growth of the cell. It tries to find the regulatory states that lead to knockouts of essential genes and deletion of essential reactions.
Note that, `find_conflicts()` results should be carefully analyzed, as this method does not detect indirect conflicts. Please consult the method for more details.

In [11]:
repressed_genes, repressed_reactions = find_conflicts(model)
repressed_genes

,interaction,b1658,b4393,b1275,ura_e,gua_e,b3938,b3828,phe_DASH_L_e,b4390,b2839,lys_DASH_L_e,leu_DASH_L_e,b0889,b3237,b3773,Stringent,b0676,tyr_DASH_L_e,b1323
b2557,b2557 || 1 = ( ~ b1658),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b1262,b1262 || 1 = ( ~ b4393),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b2752,b2752 || 1 = b1275,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b1263,b1263 || 1 = ( ~ b4393),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b2764,b2764 || 1 = b1275,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b2476,b2476 || 1 = ( ~ b1658),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b3642,b3642 || 1 = ( ~ ((ura_e > 0) | (gua_e > 0))),NaN,NaN,NaN,999999.0,999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b2424,b2424 || 1 = b1275,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b4013,b4013 || 1 = (( ~ b3938) | b3828),NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b1207,b1207 || 1 = ( ~ b1658),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# steady-state RFBA. The initial state is passed in the optimize method
rfba = RFBA(model).build()
solution = rfba.optimize(initial_state=initial_state)
solution

Method,RFBA
Model,Model iMC1010 - Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Objective,BiomassEcoli
Objective value,0.7632523603559114
Status,optimal


In [10]:
# dynamic RFBA
dynamic_solution = rfba.optimize(initial_state=initial_state, dynamic=True)
dynamic_solution.solutions

{'t_0': RFBA Solution
  Objective value: 0.5476893156724354
  Status: optimal,
 't_1': RFBA Solution
  Objective value: 0.5476893156724354
  Status: optimal}

## SRFBA

In [3]:
# loading model
model = read_model(imc1010_gem_reader, imc1010_trn_reader)

# objective function
BIOMASS_ID = 'BiomassEcoli'
model.objective = {BIOMASS_ID: 1}

SRFBA does not need an initial state in most cases. As this method performs a steady-state simulation using MILP, the solver tries to find the regulatory state favoring reactions that contribute to faster growth rates. Regulatory variables can take values between zero and one, as they are not set initially.

In [6]:
# steady-state SRFBA
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model iMC1010 - Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Objective,BiomassEcoli
Objective value,0.8218562181811009
Status,optimal


## iFVA, iDeletions

## PROM

## CoRegFlux